<a href="https://colab.research.google.com/github/PARTHIBAN-007/LLM-FineTuning/blob/main/Qwen_2-5-7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.3.13 requires tyro, which is not installed.
unsloth-zoo 2025.3.13 requires protobuf<4.0.0, 

In [4]:
from unsloth import FastLanguageModel
import torch

model , tokenizer =  FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.15: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/106k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 4,
    target_modules = ["q_proj","k_proj","v_proj","o_proj",
                      "gate_proj","up_proj","down_proj"],
    lora_alpha  =4,
    lora_dropout = 0.05,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 2501,

)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.3.15 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [6]:
alpaca_prompt = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{}
### Input:
{}
### Response:
{}
"""

eos_token = tokenizer.eos_token

def formatting_prompt_func(examples):
  instructions = examples["instruction"]
  inputs = examples["input"]
  outputs = examples["output"]
  texts = []
  for instruction , input , output in zip(instructions , inputs , outputs):
    text = alpaca_prompt.format(instruction, input, output) + eos_token
    texts.append(text)
  return {"texts":texts,}
pass
from datasets import load_dataset

dataset = load_dataset("yahma/alpaca-cleaned",split=  "train")
dataset = dataset.map(formatting_prompt_func,batched  =True)

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments ,DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from accelerate import Accelerator

training_args = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    warmup_steps = 5,
    max_steps = 40,
    learning_rate = 2e-5,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    logging_steps = 1,
    optim = "adamw_8bit",
    weight_decay = 0.001,
    lr_scheduler_type = "linear",
    seed = 2501,
    output_dir = "outputs",
    report_to = "none"

)
accelerator = Accelerator()


trainer = SFTTrainer(
    model = model ,
    tokenizer = tokenizer ,
    train_dataset = dataset,
    max_seq_length = 2048,
    dataset_num_proc = 2,
    dataset_text_field = "texts",
    packing = False,
    formatting_func=formatting_prompt_func,
    args =  training_args,

)
trainer.accelerator = accelerator

Unsloth: Tokenizing ["texts"] (num_proc=2):   0%|          | 0/51760 [00:00<?, ? examples/s]

In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1 | Total steps = 40
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 10,092,544/7,000,000,000 (0.14% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.928900
2,1.262300
3,1.104700
4,1.276100
5,1.339200
6,1.257200
7,1.152600
8,1.049800
9,1.304700
10,1.140700


In [11]:
FastLanguageModel.for_inference(model)
input=  tokenizer(
    [
        alpaca_prompt.format(
          "Continue the fibonnaci sequence.",
          "1, 1, 2, 3, 5, 8",
          "",
        )
    ] ,return_tensors = "pt").to("cuda")

output = model.generate(**input,max_new_tokens = 250,use_cache = True)
tokenizer.batch_decode(output)



['\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n### Instruction:\nContinue the fibonnaci sequence.\n### Input:\n1, 1, 2, 3, 5, 8\n### Response:\n\n13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817, 39088169, 63245986, 102334155, 165580141,']

In [15]:
FastLanguageModel.for_inference(model)
inputs=  tokenizer(
    [
        alpaca_prompt.format(
          "Continue the fibonnaci sequence.",
          "1, 1, 2, 3, 5, 8",
          "",
        )
    ] ,return_tensors = "pt").to("cuda")


from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs , streamer = text_streamer , max_new_tokens = 250)


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Continue the fibonnaci sequence.
### Input:
1, 1, 2, 3, 5, 8
### Response:

13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817, 39088169, 63245986, 102334155, 165580141,


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [16]:
model.save_pretrained("qwen-2.5-7B")
tokenizer.save_pretrained("qwen-2.5-7B")
model.push_to_hub("Parthiban007/Qwen-2.5-7B")
tokenizer.push_to_hub("Parthiban007/Qwen-2.5-7B")

README.md:   0%|          | 0.00/595 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/40.4M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Parthiban007/Qwen-2.5-7B


No files have been modified since last commit. Skipping to prevent empty commit.
